<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=bc80d83f657ddb2123545716969c480e588690c36ba6cf76fc1829afde5ffbd5
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66


    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-19 12:24:17
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  6.17 L
Current:  1.43 C
-------------------
Today PnL: -19.98 K (-0.14%)
Current PnL: -16.95 L (-11.4%)
CY Booked + Current PnL: -5.51 L (-3.71%)
-------------------
Total profit:  2.05 L
Total loss:  -19.00 L
-------------------
Total Booked + Current PnL: 21.47 L (17.64%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.02%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.24 C
Est FTT PnL: 81.47 L (57.11%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 9.83%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,13.31,63.0,H-MC,3.48,195460.0,30790.0,12549.0,-0.33,18.70,6.42,26.31,89.0,2.45,1.43,28.17,X40N,NTT,AC
76,TTKPRESTIG,770.00,102.77,52.0,M-SC,3.71,87325.0,-13452.0,13544.0,-1.04,-13.35,15.51,0.09,245.0,-0.99,0.64,14.73,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,1.72,60.0,H-MC,2.69,213300.0,21558.0,16189.0,-0.29,11.24,7.59,19.69,99.0,1.33,1.56,18.80,XY25,NTT,AC
49,NATIONALUM,244.55,-43.66,66.0,H-MC,7.96,112323.0,11819.0,17287.0,0.30,11.76,15.39,28.96,79.0,0.68,0.82,49.72,MH,ATH,METALS
56,RELIANCE,1533.00,-12.16,57.0,H-LC,4.95,219851.0,9485.0,19303.0,-0.40,4.51,8.78,13.68,37.0,0.49,1.61,21.78,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.0,-175.00,72.0,M-SC,3.35,86766.0,-15444.0,108831.0,3.89,-15.11,125.43,91.37,208.0,-0.14,0.64,40.85,XY25,NTT,FINANCE
78,UNIONBANK,163.0,-11.63,68.0,M-LC,7.07,160146.0,19306.0,25191.0,3.12,13.71,15.73,31.59,66.0,0.77,1.17,43.83,XY24,NTT,BANKS
59,SAIL,228.0,48.36,70.0,M-MC,7.55,233934.0,8972.0,158677.0,2.00,3.99,67.83,74.53,192.0,0.06,1.71,37.53,XY24,BTT,STEEL
6,ASIANTILES,137.0,7088.89,67.0,L-SC,5.23,78870.0,-14940.0,91560.0,1.68,-15.93,116.09,81.67,269.0,-0.16,0.58,52.73,XR,NTT,CERAMICS
11,BANKINDIA,190.0,-27.95,67.0,H-MC,6.36,185929.0,6121.0,106296.0,1.58,3.40,57.17,62.52,88.0,0.06,1.36,38.65,XR,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,QUESS,986.00,-41.60,44.0,M-SC,39.87,58243.0,-6763.0,157693.0,-2.44,-10.40,270.75,232.18,198.0,-0.04,0.43,1.43,XY24,NTT,MISC
80,VAIBHAVGBL,521.00,61.98,65.0,H-SC,6.64,140368.0,-42407.0,153478.0,-2.29,-23.20,109.34,60.77,125.0,-0.28,1.03,25.72,XR,NTT,JEWELLERY
36,INDIAMART,4850.92,-53.15,34.0,H-SC,11.48,130910.0,7574.0,126184.0,-2.10,6.14,96.39,108.45,119.0,0.06,0.96,29.87,AR,ATH,MISC
33,ICICIGI,2260.25,-20.11,45.0,H-MC,2.30,137285.0,1152.0,29969.0,-1.61,0.85,21.83,22.86,91.0,0.04,1.01,13.57,X40,ATH,INSURANCE
55,RELAXO,1176.00,-32.71,53.0,H-SC,1.93,91669.0,-53491.0,129418.0,-1.57,-36.85,141.18,52.31,136.0,-0.41,0.67,22.70,X40N,NTT,FOOTWEAR


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,IEX,219.00,-29.46,57.0,H-SC,16.92,201745.0,1791.0,95446.0,0.04,0.90,47.31,48.63,115.0,0.02,1.48,12.55,XR,NTT,MISC
34,ICICIPRULI,790.00,-20.54,47.0,H-MC,2.18,136786.0,998.0,41747.0,0.04,0.74,30.52,31.48,107.0,0.02,1.00,12.94,X40,ATH,INSURANCE
29,HAVELLS,2069.16,0.62,60.0,H-MC,1.93,249460.0,1729.0,73341.0,0.30,0.70,29.40,30.30,92.0,0.02,1.83,14.08,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-20.11,45.0,H-MC,2.30,137285.0,1152.0,29969.0,-1.61,0.85,21.83,22.86,91.0,0.04,1.01,13.57,X40,ATH,INSURANCE
51,PGHH,17973.08,-30.81,52.0,H-MC,4.09,200085.0,-735.0,69510.0,-0.10,-0.37,34.74,34.25,80.0,-0.01,1.47,4.81,X40,ATH,FMCG


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,96.03,55.0,H-SC,9.43,123930.0,-18549.0,38071.0,0.86,-13.02,30.72,13.70,163.0,-0.49,0.91,46.27,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,102.77,52.0,M-SC,3.71,87325.0,-13452.0,13544.0,-1.04,-13.35,15.51,0.09,245.0,-0.99,0.64,14.73,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.06,62.0,H-SC,1.26,226800.0,-42867.0,79199.0,-0.49,-15.90,34.92,13.47,138.0,-0.54,1.66,14.71,XY24,NTT,PAINTS
18,CERA,9475.0,-20.05,45.0,H-SC,2.10,145567.0,-30336.0,72361.0,-0.29,-17.25,49.71,23.89,149.0,-0.42,1.07,25.72,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-20.01,52.0,H-MC,7.46,108310.0,-23525.0,65755.0,-0.58,-17.84,60.71,32.03,98.0,-0.36,0.79,21.32,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,48.36,70.0,M-MC,7.55,233934.0,8972.0,158677.0,2.00,3.99,67.83,74.53,192.0,0.06,1.71,37.53,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,13.31,63.0,H-MC,3.48,195460.0,30790.0,12549.0,-0.33,18.70,6.42,26.31,89.0,2.45,1.43,28.17,X40N,NTT,AC
83,VOLTAS,1530.00,1.72,60.0,H-MC,2.69,213300.0,21558.0,16189.0,-0.29,11.24,7.59,19.69,99.0,1.33,1.56,18.80,XY25,NTT,AC
17,CAMS,5211.76,1.71,69.0,H-SC,3.44,115556.0,13552.0,30368.0,1.27,13.29,26.28,43.06,122.0,0.45,0.85,32.68,X40N,ATH,MISC
29,HAVELLS,2069.16,0.62,60.0,H-MC,1.93,249460.0,1729.0,73341.0,0.30,0.70,29.40,30.30,92.0,0.02,1.83,14.08,X40,ATH,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,244.55,-43.66,66.0,H-MC,7.96,112323.0,11819.0,17287.0,0.30,11.76,15.39,28.96,79.0,0.68,0.82,49.72,MH,ATH,METALS
84,WHIRLPOOL,2270.00,-41.04,49.0,M-SC,0.64,99712.0,8214.0,70536.0,-0.44,8.98,70.74,86.07,223.0,0.12,0.73,44.80,XR,NTT,DURABLES
85,WIPRO,420.00,-10.16,57.0,M-LC,5.66,158267.0,7322.0,102130.0,-0.65,4.85,64.53,72.51,53.0,0.07,1.16,11.17,XR,NTT,IT
11,BANKINDIA,190.00,-27.95,67.0,H-MC,6.36,185929.0,6121.0,106296.0,1.58,3.40,57.17,62.52,88.0,0.06,1.36,38.65,XR,NTT,BANKS
36,INDIAMART,4850.92,-53.15,34.0,H-SC,11.48,130910.0,7574.0,126184.0,-2.10,6.14,96.39,108.45,119.0,0.06,0.96,29.87,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VBL,671.64,-16.28,42.0,H-LC,6.57,301774.0,-14068.0,126745.0,-0.12,-4.45,42.00,35.67,5.0,-0.11,2.21,8.78,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-12.67,49.0,H-LC,7.80,220422.0,-31346.0,84554.0,1.05,-12.45,38.36,21.13,27.0,-0.37,1.62,18.56,X40,ATH,PAINTS
41,ITC,452.00,-37.92,49.0,H-LC,2.38,200539.0,401.0,20495.0,-0.35,0.20,10.22,10.44,4.0,0.02,1.47,5.77,X40,NTT,FMCG
30,HCLTECH,1943.91,1.08,49.0,H-LC,9.73,231167.0,-10757.0,74020.0,-1.43,-4.45,32.02,26.15,8.0,-0.15,1.69,9.24,X40,ATH,IT
79,UNITDSPR,1644.00,-13.23,50.0,H-LC,7.78,227608.0,-6536.0,55149.0,-0.31,-2.79,24.23,20.77,86.0,-0.12,1.67,3.93,X40N,NTT,BREWERIES


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,0.00,69.0,H-LC,1.48,248198.0,2047.0,77165.0,-0.16,0.83,31.09,32.18,16.0,0.03,1.82,37.17,X200,ATH,IT
41,ITC,452.0,-37.92,49.0,H-LC,2.38,200539.0,401.0,20495.0,-0.35,0.20,10.22,10.44,4.0,0.02,1.47,5.77,X40,NTT,FMCG
50,NESTLEIND,1377.0,-9.11,52.0,H-LC,4.18,279583.0,14157.0,42636.0,-1.18,5.33,15.25,21.40,11.0,0.33,2.05,13.06,XY25,NTT,FMCG
56,RELIANCE,1533.0,-12.16,57.0,H-LC,4.95,219851.0,9485.0,19303.0,-0.40,4.51,8.78,13.68,37.0,0.49,1.61,21.78,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-19.78,51.0,H-LC,5.71,213912.0,9412.0,32001.0,-1.06,4.60,14.96,20.25,10.0,0.29,1.57,13.74,X40N,BTT,PHARMA


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNITDSPR,1644.00,-13.23,50.0,H-LC,7.78,227608.0,-6536.0,55149.0,-0.31,-2.79,24.23,20.77,86.0,-0.12,1.67,3.93,X40N,NTT,BREWERIES
73,TCS,4476.75,-25.47,56.0,H-LC,12.30,296589.0,-49367.0,124211.0,-0.68,-14.27,41.88,21.64,1.0,-0.40,2.17,5.07,X40,ATH,IT
41,ITC,452.00,-37.92,49.0,H-LC,2.38,200539.0,401.0,20495.0,-0.35,0.20,10.22,10.44,4.0,0.02,1.47,5.77,X40,NTT,FMCG
82,VBL,671.64,-16.28,42.0,H-LC,6.57,301774.0,-14068.0,126745.0,-0.12,-4.45,42.00,35.67,5.0,-0.11,2.21,8.78,X40N,ATH,FMCG
30,HCLTECH,1943.91,1.08,49.0,H-LC,9.73,231167.0,-10757.0,74020.0,-1.43,-4.45,32.02,26.15,8.0,-0.15,1.69,9.24,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,7969.85,3.38,64.0,H-LC,26.99,166015.0,-20080.0,232471.0,0.08,-10.79,140.03,114.13,15.0,-0.09,1.22,22.26,AR,ATH,ELECTRICAL
41,ITC,452.00,-37.92,49.0,H-LC,2.38,200539.0,401.0,20495.0,-0.35,0.20,10.22,10.44,4.0,0.02,1.47,5.77,X40,NTT,FMCG
19,CIPLA,1795.00,-19.78,51.0,H-LC,5.71,213912.0,9412.0,32001.0,-1.06,4.60,14.96,20.25,10.0,0.29,1.57,13.74,X40N,BTT,PHARMA
56,RELIANCE,1533.00,-12.16,57.0,H-LC,4.95,219851.0,9485.0,19303.0,-0.40,4.51,8.78,13.68,37.0,0.49,1.61,21.78,XY25,NTT,REFINERIES
5,ASIANPAINT,3465.66,-12.67,49.0,H-LC,7.80,220422.0,-31346.0,84554.0,1.05,-12.45,38.36,21.13,27.0,-0.37,1.62,18.56,X40,ATH,PAINTS


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-25.47,56.0,H-LC,12.30,296589.0,-49367.0,124211.0,-0.68,-14.27,41.88,21.64,1.0,-0.40,2.17,5.07,X40,ATH,IT
39,INFY,2275.00,-16.73,54.0,H-LC,8.20,324229.0,11341.0,160331.0,-1.19,3.62,49.45,54.87,3.0,0.07,2.38,10.51,X40,BTT,IT
41,ITC,452.00,-37.92,49.0,H-LC,2.38,200539.0,401.0,20495.0,-0.35,0.20,10.22,10.44,4.0,0.02,1.47,5.77,X40,NTT,FMCG
82,VBL,671.64,-16.28,42.0,H-LC,6.57,301774.0,-14068.0,126745.0,-0.12,-4.45,42.00,35.67,5.0,-0.11,2.21,8.78,X40N,ATH,FMCG
1,ABB,7934.00,-37.53,68.0,H-LC,9.85,261120.0,-499.0,119724.0,0.07,-0.19,45.85,45.57,7.0,-0.00,1.91,11.11,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7088.89,67.0,L-SC,5.23,78870.0,-14940.0,91560.0,1.68,-15.93,116.09,81.67,269.0,-0.16,0.58,52.73,XR,NTT,CERAMICS
21,COFFEEDAY,80.00,-41.65,52.0,L-SC,42.28,85152.0,-28397.0,68445.0,0.89,-25.01,80.38,35.27,268.0,-0.41,0.62,108.41,XR,NTT,HOTELS
60,SAMMAANCAP,326.00,-175.00,72.0,M-SC,3.35,86766.0,-15444.0,108831.0,3.89,-15.11,125.43,91.37,208.0,-0.14,0.64,40.85,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-4.17,48.0,H-SC,12.50,89182.0,-11781.0,106314.0,-0.06,-11.67,119.21,93.63,148.0,-0.11,0.65,32.83,SR,ATH,CHEMICALS
48,MASFIN,398.61,-16.44,53.0,H-SC,12.99,94785.0,-3195.0,24796.0,0.59,-3.26,26.16,22.05,152.0,-0.13,0.69,37.78,XR,ATH,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1102.55,69.0,M-SC,4.57,163624.0,-17519.0,62979.0,0.34,-9.67,38.49,25.10,235.0,-0.28,1.20,31.13,XY24,NTT,HEALTHCARE
60,SAMMAANCAP,326.00,-175.00,72.0,M-SC,3.35,86766.0,-15444.0,108831.0,3.89,-15.11,125.43,91.37,208.0,-0.14,0.64,40.85,XY25,NTT,FINANCE
7,ATULAUTO,844.00,3845.95,61.0,M-SC,5.35,123050.0,-24318.0,76131.0,-1.28,-16.50,61.87,35.16,236.0,-0.32,0.90,25.68,XY24,NTT,AUTO
74,TITAGARH,1548.00,0.96,68.0,H-SC,3.16,165454.0,-26561.0,105444.0,-0.11,-13.83,63.73,41.08,158.0,-0.25,1.21,40.20,XY24,NTT,ENGINEERING
70,TANLA,1963.11,-19.10,71.0,H-SC,12.14,217153.0,-53608.0,383557.0,1.44,-19.80,176.63,121.86,133.0,-0.14,1.59,69.87,AR,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.06
1,25,43.90
2,50,74.47


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.54
LC    32.11
MC    23.32
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.33
X40      15.39
X40N     11.95
XR       11.80
AR        8.76
XY25      8.28
OX40N     8.21
X200      1.82
MH        1.70
X5K       1.44
SR        1.29
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.30
H-LC    25.39
H-MC    20.30
M-SC    13.65
M-LC     5.65
M-MC     2.71
L-SC     1.59
L-LC     1.07
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.63,-12.98,74.58
FMCG,11.89,-4.41,40.94
FINANCE,9.79,-15.31,60.47
MISC,7.46,-13.01,75.84
BANKS,6.37,-14.59,74.87
PAINTS,5.87,-15.17,32.34
ELECTRICAL,5.78,-4.35,61.87
AC,3.71,4.56,11.42
INSURANCE,3.44,-7.24,44.03


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2839572.0,22
AR,1300555.0,9
XR,1298365.0,14
X40,749103.0,10
X40N,574350.0,9
OX40N,526149.0,10
XY25,335136.0,6
SR,244816.0,2
X5K,120682.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3412842.0,29
M-SC,1381322.0,17
H-LC,1229090.0,14
H-MC,1096454.0,15
M-LC,377352.0,4
M-MC,307284.0,2
L-SC,243610.0,3
L-MC,60415.0,1
L-LC,38437.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1025350.0      6
M-SC       XY24       799864.0      7
H-SC       AR         787232.0      5
           XR         754747.0      7
H-MC       XY24       492249.0      4
H-LC       X40        463611.0      5
           AR         352195.0      2
M-MC       XY24       307284.0      2
H-SC       X40N       286084.0      4
           OX40N      250987.0      4
           SR         244816.0      2
H-MC       X40        214567.0      4
H-LC       X40N       213895.0      3
M-SC       AR         161128.0      2
L-SC       XR         160005.0      2
M-LC       XY24       154540.0      2
H-MC       XY25       125929.0      2
M-SC       OX40N      125802.0      4
M-LC       X5K        120682.0      1
M-SC       XR         114772.0      2
           XY25       108831.0      1
H-MC       XR         106296.0      1
M-LC       XR         102130.0      1
L-SC       OX40N       83605.0      1
H-LC       X200        77165.0      1
H-MC       X40N        74371.0      2
M-SC       X40         70925.0      1
H-MC       OX40N       65755.0      1
H-SC       MH          63626.0      1
H-LC       XY25        61939.0      2
L-MC       XR          60415.0      1
H-LC       XY24        60285.0      1
L-LC       XY25        38437.0      1
H-MC       MH          17287.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 54.0 seconds
